In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
base_dir: bool = False

In [2]:
import os, sys

if not base_dir:
    os.chdir('../')
    base_dir = True

import numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn, torch

from torch.utils.data import DataLoader, TensorDataset

import torch.nn as nn

from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score

from utils.data_tools import inbreast, cbis_ddsm, image_processing
from utils.models import models

import tensorflow as tf

In [4]:
inbreast.download()
cbis_ddsm.download()

Path already exists for INBreast!
Path already exists for CBIS-DDSM!


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
to_unique = tf.keras.utils.to_categorical

In [5]:
train, test = (x_train.reshape((60000, 28, 28, 1)), to_unique(y_train).reshape((60000, 10, 1))), (x_test.reshape((10000, 28, 28, 1)), to_unique(y_test).reshape((10000, 10, 1)))

In [3]:
def load_mnist(batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

    # Convert grayscale to 3-channel
    train_data = train_dataset.data.unsqueeze(1).repeat(1, 3, 1, 1).float()
    test_data = test_dataset.data.unsqueeze(1).repeat(1, 3, 1, 1).float()

    num_classes = 10
    train_targets = nn.functional.one_hot(train_dataset.targets, num_classes=num_classes).float()
    test_targets = nn.functional.one_hot(test_dataset.targets, num_classes=num_classes).float()

    train_loader = DataLoader(TensorDataset(train_data, train_targets), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(test_data, test_targets), batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


In [4]:
M = models((load_mnist()))

In [5]:
cnn = M.SmallCNN_torch()

c:\Users\Theon\miniconda3\Lib\site-packages\torch\nn\modules\conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Convolution.cpp:1037.)
  return F.conv2d(


Epoch 1/10, Loss: 14.1599
Epoch 2/10, Loss: 14.1600


KeyboardInterrupt: 

In [14]:
train[0].shape

(60000, 28, 28, 1)

In [10]:
# Get training features from cbis_ddsm and validation data from inbreast

class cbis_ddsm_data:

    x_train, x_test, y_train, y_test = data = cbis_ddsm.get_images()

class inbreast_data:

    x_train, x_test, y_train, y_test = data = inbreast.get_images()

    x, y = np.concat([x_train, x_test]), np.concat([y_train, y_test])

KeyboardInterrupt: 

In [30]:
# Create model wrapper

M = models(cbis_ddsm_data.data)

In [31]:
# Contruct and Train model
cnn = M.SmallCNN(epochs=3, lr=0.01, metrics=['auc', 'accuracy'])

Epoch 1/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 153ms/step - accuracy: 0.7988 - auc: 0.8467 - loss: 0.9398
Epoch 2/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 148ms/step - accuracy: 0.9798 - auc: 0.9928 - loss: 0.0860
Epoch 3/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 148ms/step - accuracy: 0.9813 - auc: 0.9908 - loss: 0.0890
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9802 - auc: 0.9985 - loss: 0.0595

            SmallCNN stats

            BinaryCrossentropy Loss: 0.04648654907941818
            auc: 0.9989706873893738
accuracy: 0.9842657446861267
            


In [32]:
# Validate on INBreast
y_pred = [np.round(p) for p in cnn.predict(inbreast_data.x_test, verbose=0)]

print(inbreast_accuracy := accuracy_score(inbreast_data.y_test, y_pred))

0.9878048780487805


In [33]:
# Model information

from timeit import timeit

def predict_100_images() -> None:
    cnn.predict(cbis_ddsm_data.x_test[: 10], verbose=0)

print(f"""
Validation Accuracy (INBreast): {round(inbreast_accuracy, 2) * 100}%
Number of Params: {(params := cnn.count_params())}
Memory Allocation: {round((params * 4) / (1024**2), 2)} MB
Run time (100 images GPU): {timeit(predict_100_images, number=1):.2f} sec
""")


Validation Accuracy (INBreast): 99.0%
Number of Params: 161313
Memory Allocation: 0.62 MB
Run time (100 images GPU): 0.08 sec



In [35]:
cnn.save('models/SmallCNN.keras')

In [ ]:
# GradCAM implementation

# from utils.grad_cam import GradCAM_Tools

# GradCAM_Tools.display.GradCAM(cnn, inbreast_data.x[0])